# HW 3
- Authors: Marc Brooks, Jack McCarthy, Michael Sarkis
- NetID: mgb45, ___ , ms939

## 1.) (Hard-thresholding)

(a)
\begin{aligned}
\mathbb{E}Z\textbf{1}_{\left| Z \right| \le \gamma}
&= \int_{-\infty}^{\infty} \frac{z}{\tau \sqrt{2 \pi}} e^{-\frac{1}{2}\left( \frac{z-\omega}{\tau} \right)^2}\textbf{1}_{\left| Z \right| \le \gamma} dz \\
&= \frac{1}{\sqrt{2 \pi}}\int_{-\gamma}^{\gamma} \frac{z}{\tau} e^{-\frac{1}{2}\left( \frac{z-\omega}{\tau} \right)^2} dz
\end{aligned}
Let $u = \frac{z-\omega}{\tau}$, so $z = u \tau + \omega$ and $du = \frac{1}{\tau}dz$. Substituting these in we get the following:
\begin{aligned}
\frac{1}{\sqrt{2 \pi}}\int_{-\gamma}^{\gamma} \frac{z}{\tau} e^{-\frac{1}{2}\left( \frac{z-\omega}{\tau} \right)^2} dz
&= \frac{1}{\sqrt{2 \pi}} \left[ \int (u \tau + \omega)e^{-\frac{1}{2}u^2}du \right]_{-\frac{\gamma + \omega}{\tau}}^{\frac{\gamma - \omega}{\tau}} \\
&= \frac{1}{\sqrt{2 \pi}} \left[ \tau \int ue^{-\frac{1}{2}u^2}du + \omega \int e^{-\frac{1}{2}u^2}du \right]_{-\frac{\gamma + \omega}{\tau}}^{\frac{\gamma - \omega}{\tau}} \\
&= \frac{1}{\sqrt{2 \pi}} \left[ -\tau e^{-\frac{1}{2} u^2} + \frac{\omega \sqrt{\pi}}{\sqrt{2}} \text{erf} \left( \frac{u}{\sqrt{2}} \right) \right]_{-\frac{\gamma + \omega}{\tau}}^{\frac{\gamma - \omega}{\tau}} \\
&= \left[ -\frac{\tau}{\sqrt{2 \pi}} e^{-\frac{1}{2}u^2} + \frac{\omega}{2} \text{erf} \left( \frac{u}{\sqrt{2}} \right) \right]_{-\frac{\gamma + \omega}{\tau}}^{\frac{\gamma - \omega}{\tau}}
\end{aligned}
Which evaluated gives:
$$\mathbb{E}Z\textbf{1}_{\left| Z \right| \le \gamma} = \frac{\omega}{2} \left[ \text{erf} \left( \frac{\gamma-\omega}{\tau \sqrt{2}} \right) - \text{erf} \left( \frac{-(\gamma+\omega)}{\tau \sqrt{2}} \right) \right] - \frac{\tau}{\sqrt{2 \pi}} \left[ \exp\left\{-\frac{1}{2}\left( \frac{\gamma-\omega}{\tau} \right)^2\right\} - \exp\left\{-\frac{1}{2}\left( \frac{-(\gamma+\omega)}{\tau} \right)^2\right\} \right]$$
(b)
\begin{aligned}
\mathbb{E}\left( X - \hat{\mu}_n^H \right)^2
           &= \mathbb{E}\left( (X - \mu) + (\mu - \hat{\mu}_n^H) \right)^2 \\
           &= \mathbb{E}(X - \mu)^2 + \mathbb{E}(\mu - \hat{\mu}_n^H)^2 \\
           &= \sigma^2 + \mathbb{E}(\mu - \hat{\mu}_n^H) \\
           &= \sigma^2 + \mathbb{E}(\mu^2 - 2\mu \hat{\mu}_n^H + {\hat{\mu}_n^H}^2) \\
           &= \sigma^2 + \mu^2 - 2\mu (\mathbb{E}\bar{X}_n - \mathbb{E}\bar{X}_n \textbf{1}_{\left| \bar{X}_n \right| \le \alpha}) + \mathbb{E}{\hat{\mu}_n^H}^2 \\
           &= \sigma^2 +\mu^2 - 2\mu^2 + 2\mu \mathbb{E}\bar{X}_n \textbf{1}_{\left| \bar{X}_n \right| \le \alpha} + \mathbb{E}{\hat{\mu}_n^H}^2 \\
           &= \sigma^2 - \mu^2 + 2\mu \mathbb{E}\bar{X}_n \textbf{1}_{\left| \bar{X}_n \right| \le \alpha} + \mathbb{E}{\hat{\mu}_n^H}^2
\end{aligned}
We can plug the closed form from (a) in for $\mathbb{E}\bar{X}_n \textbf{1}_{\left| \bar{X}_n \right| \le \alpha}$ and evaluate the second moment of $\bar{X}_n\textbf{1}_{\left| \bar{X}_n \right| \ge \alpha}$ to obtain the final expression. To solve for $\alpha^{opt}$, take the derivative with respect to $\alpha$ and set the resulting expression $= 0$. One can also perform a computational search across a range of $\alpha$ values to identify an approximate optimal value.

(c)

A possible estimator of $\alpha^{opt}$ would be
$$ \hat{\alpha}_n = \underset{\alpha}{\arg \min} (\text{closed form of} \; \mathbb{E}\left( X - \hat{\mu}_n^H \right)^2)  $$
where $\mu$ and $\sigma^2$ are replaced with the sample mean and variance.

## 2. (Great British Bake-off)

In [2]:
# importing required packages
import numpy as np
import pandas as pd
import itertools

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoLarsIC, LassoLarsCV, RidgeCV, LinearRegression, Ridge
from sklearn.model_selection import cross_val_score, LeaveOneOut

import warnings
warnings.filterwarnings("ignore")

## (a)

First, since $\epsilon$ and $\textbf{X}$ are independent we can see that: 
$$
\begin{align}
\text{Var}(Y) &= \text{Var}(\textbf{X}^{T}\beta^{*} + \epsilon) \\
&= \text{Var}(\textbf{X}^{T}\beta^{*}) + \text{Var}(\epsilon) \\
&= \beta^{*T}\Sigma(\rho)\beta^{*} + \sigma^2 
\end{align}
$$

Furthmore, we recognize $P(Y - \textbf{X}^{T}\beta^{*})^{2} = P\epsilon^{2}$ as the expected squared residuals.
Since $P\epsilon = 0$ then $P\epsilon^2 = \sigma^2$ and $P(Y - \textbf{X}^{T}\beta^{*})^{2} = \sigma^2$.


Thus,
$$
\begin{align}
R^{2} &= 1 - \frac{P(Y - \textbf{X}^{T}\beta^{*})^{2}}{\text{Var}(Y)} \\
&= 1 - \frac{\sigma^2}{\beta^{*T}\Sigma(\rho)\beta^{*} + \sigma^2}
\end{align}
$$

## (b)

Setting $R^2 = .8$ we can solve for $\sigma^2$ as follows.

$$
\begin{align}
.8 &= 1 - \frac{\sigma^2}{\beta^{*T}\Sigma(\rho)\beta^{*} + \sigma^2} \\
.2 &= \frac{\sigma^2}{\beta^{*T}\Sigma(\rho)\beta^{*} + \sigma^2} \\
\sigma^2 &= .2(\beta^{*T}\Sigma(\rho)\beta^{*} + \sigma^2)  \\
.8\sigma^2 &= .2(\beta^{*T}\Sigma(\rho)\beta^{*})  \\
\sigma^2 &= .25(\beta^{*T}\Sigma(\rho)\beta^{*}) 
\end{align}
$$

In [3]:
labels = ['lasso']*3 + ['adpt lasso']*3 + ['ridge']*3 + ['adpt ridge']*3
tunings = ['AIC', 'BIC', 'LOO-CV']*4
labs = [labels[i] + ' ' + tunings[i] for i in range(len(labels))]

In [6]:
emse_table = pd.DataFrame(itertools.product(['sparse', 'dense'], 
                              labs,
                               [10,25,50],
                               [0,.25,.5]), 
             columns=['signal','Method-Tuning', 'p', 'rho'])

emse_table['mse'] = None

In [34]:
def calc_bic(y_hat, y, D, lamb):
    df_l = np.sum(D**2/(D**2 + lamb))
    rss = np.sum((y - y_hat)**2)
    bic = np.log(rss) + df_l*np.log(len(y))/len(y)
    return bic

In [35]:
def calc_aic(y_hat, y, D, lamb):
    df_l = np.sum(D**2/(D**2 + lamb))
    rss = np.sum((y.ravel() - y_hat.ravel())**2)
    aic = np.log(rss) + df_l*2/len(y)
    return aic

In [36]:
def lambda_ic(lambs, X, Y, X_star, Y_star, ic='aic'):
    U, D, V = np.linalg.svd(X, full_matrices=False)
    y_pred = lambda l: (U @ np.diag(D**2/(D**2 + l)) @ U.T) @ Y   
    if ic=='bic':
        bic_vals = np.array([calc_bic(y_hat = y_pred(l), y=Y, D=D, lamb=l) for l in lambs])
        min_i = np.argmin(bic_vals)
    else:
        aic_vals = np.array([calc_aic(y_hat = y_pred(l), y=Y, D=D, lamb=l) for l in lambs])
        min_i = np.argmin(aic_vals)
    mse = mean_squared_error(Y, y_pred(lambs[min_i]))
    #beta_hat = np.linalg.inv(X.T @ X + lambs[min_i]*np.eye(p)) @ X.T @ Y
    ridge = Ridge(alpha=lambs[i], fit_intercept=False)
    #omse = mean_squared_error(Y_star, X_star @ beta_hat)
    ridge.fit(X,Y)
    omse = mean_squared_error(Y_star, ridge.predict(X_star))
    return mse, lambs[min_i], omse

In [40]:
EMSES = np.zeros((24,9))

In [64]:
from tqdm import tqdm

datasets = 25 #number of loop iterations
n = 100
count = 0
test_size = 1000

# Sparse Signal
lasso_aic_mse_spr = np.zeros(9)
lasso_bic_mse_spr = np.zeros(9)
lasso_loocv_mse_spr = np.zeros(9)
adp_lasso_aic_mse_spr = np.zeros(9)
adp_lasso_bic_mse_spr = np.zeros(9)
adp_lasso_loocv_mse_spr = np.zeros(9)

ridge_aic_mse_spr = np.zeros(9)
ridge_bic_mse_spr = np.zeros(9)
ridge_loocv_mse_spr = np.zeros(9)
adp_ridge_aic_mse_spr = np.zeros(9)
adp_ridge_bic_mse_spr = np.zeros(9)
adp_ridge_loocv_mse_spr =np.zeros(9)

# Dense Signal 
lasso_aic_mse_dns = np.zeros(9)
lasso_bic_mse_dns = np.zeros(9)
lasso_loocv_mse_dns = np.zeros(9)
adp_lasso_aic_mse_dns = np.zeros(9)
adp_lasso_bic_mse_dns = np.zeros(9)
adp_lasso_loocv_mse_dns = np.zeros(9)

ridge_aic_mse_dns = np.zeros(9)
ridge_bic_mse_dns = np.zeros(9)
ridge_loocv_mse_dns = np.zeros(9)
adp_ridge_aic_mse_dns = np.zeros(9)
adp_ridge_bic_mse_dns = np.zeros(9)
adp_ridge_loocv_mse_dns = np.zeros(9)
    
for i,(p,rho) in enumerate(itertools.product([10, 25, 50],[0, 0.25, 0.5])):
    print(p, rho)
    
    # Generate data
    beta_spr = np.fromfunction(lambda j, _: (2 / np.sqrt(n)) * (j + 1 <= np.sqrt(p)), (p, 1))
    beta_dns = np.fromfunction(lambda j, _: (5 / ((j + 1) * np.sqrt(n))), (p, 1))

    Sigma_p = np.fromfunction(lambda i, j: rho**(np.abs(i - j)), (p, p))
    sigma2_spr = 0.25 * (beta_spr.T @ Sigma_p @ beta_spr)
    sigma2_dns = 0.25 * (beta_dns.T @ Sigma_p @ beta_dns)
    
    
    #Generate Test sets for given Rho and p
    test_epsilon_spr = np.random.normal(loc=0, scale=np.sqrt(sigma2_spr))
    test_epsilon_dns = np.random.normal(loc=0, scale=np.sqrt(sigma2_dns))
    
    test_X = np.random.multivariate_normal(mean=np.zeros(p), cov=Sigma_p, size=test_size)

    test_Y_spr =  test_X @ beta_spr + test_epsilon_spr
    test_Y_dns =  test_X @ beta_dns + test_epsilon_dns
    
    
    # For LOO-CV
    loo = LeaveOneOut()
    
    
    
    for _ in range(datasets):
        
        #To divide out mse sums later
        count += 1 
        
        # Create Dataset
        epsilon_spr = np.random.normal(loc=0, scale=np.sqrt(sigma2_spr))
        epsilon_dns = np.random.normal(loc=0, scale=np.sqrt(sigma2_dns))

        X = np.random.multivariate_normal(mean=np.zeros(p), cov=Sigma_p, size=n)

        Y_spr =  X @ beta_spr + epsilon_spr
        Y_dns =  X @ beta_dns + epsilon_dns

        # For adaptive regressions2.289212 	26.952930 	0.177174
        lm_spr = LinearRegression().fit(X, Y_spr)
        lm_dns = LinearRegression().fit(X, Y_dns)
        
        # Adpative Lasso
        # Here we are compute X^T (D^-1)^T
        adX_sprL = X @ np.diagflat(np.abs(lm_spr.coef_))
        adX_dnsL = X @ np.diagflat(np.abs(lm_dns.coef_))
        
        # Adpative Ridge
        adX_sprR = X @ np.diagflat((lm_spr.coef_**2)**(1/2))
        adX_dnsR = X @ np.diagflat((lm_dns.coef_**2)**(1/2))
        
        # Adpative Lasso
        # Here we are compute X^T (D^-1)^T
        test_adX_sprL = test_X @ np.diagflat(np.abs(lm_spr.coef_))
        test_adX_dnsL = test_X @ np.diagflat(np.abs(lm_dns.coef_))
        
        # Adpative Ridge
        test_adX_sprR = test_X @ np.diagflat((lm_spr.coef_**2)**(1/2))
        test_adX_dnsR = test_X @ np.diagflat((lm_dns.coef_**2)**(1/2))
    
    
        # Calculate Lassos
        
        # AIC
        
        # Sparse
        lasso_aic_spr = LassoLarsIC(criterion='aic')
        lasso_aic_spr.fit(X, Y_spr)
        lasso_aic_mse_spr[i] += mean_squared_error(test_Y_spr, lasso_aic_spr.predict(test_X))
        
        # Dense
        lasso_aic_dns = LassoLarsIC(criterion='aic')
        lasso_aic_dns.fit(X, Y_dns)
        lasso_aic_mse_dns[i] += mean_squared_error(test_Y_dns, lasso_aic_dns.predict(test_X))
        
        #BIC
        
        # Sparse
        lasso_bic_spr = LassoLarsIC(criterion='bic')
        lasso_bic_spr.fit(X, Y_spr)
        lasso_bic_mse_spr[i] += mean_squared_error(test_Y_spr, lasso_bic_spr.predict(test_X))
        
        # Dense
        lasso_bic_dns = LassoLarsIC(criterion='bic')
        lasso_bic_dns.fit(X, Y_dns)
        lasso_bic_mse_dns[i] += mean_squared_error(test_Y_dns, lasso_bic_dns.predict(test_X))
        
        #LOO-CV
        
        # Sparse
        lasso_loo_spr = LassoLarsCV(cv=loo)
        lasso_loo_spr.fit(X, Y_spr)
        lasso_loocv_mse_spr[i] += mean_squared_error(test_Y_spr, lasso_loo_spr.predict(test_X))
        
        # Dense
        lasso_loo_dns = LassoLarsCV(cv=loo)
        lasso_loo_dns.fit(X, Y_dns)
        lasso_loocv_mse_dns[i] += mean_squared_error(test_Y_dns, lasso_loo_dns.predict(test_X))

        # Adaptive Lasso
        
        # AIC
        
        # Sparse
        adp_lasso_aic_spr = LassoLarsIC(criterion='aic')
        adp_lasso_aic_spr.fit(adX_sprL, Y_spr)
        adp_lasso_aic_mse_spr[i] += mean_squared_error(test_Y_spr, adp_lasso_aic_spr.predict(test_adX_sprL))
        
        # Dense
        adp_lasso_aic_dns = LassoLarsIC(criterion='aic')
        adp_lasso_aic_dns.fit(adX_dnsL, Y_dns)
        adp_lasso_aic_mse_dns[i] += mean_squared_error(test_Y_dns, adp_lasso_aic_dns.predict(test_adX_dnsL))
        
        # BIC
        
        # Sparse
        adp_lasso_bic_spr = LassoLarsIC(criterion='bic')
        adp_lasso_bic_spr.fit(adX_sprL, Y_spr)
        adp_lasso_bic_mse_spr[i] += mean_squared_error(test_Y_spr, adp_lasso_bic_spr.predict(test_adX_sprL))
        
        # Dense
        adp_lasso_bic_dns = LassoLarsIC(criterion='bic')
        adp_lasso_bic_dns.fit(adX_dnsL, Y_dns)
        adp_lasso_bic_mse_dns[i] += mean_squared_error(test_Y_dns, adp_lasso_bic_dns.predict(test_adX_dnsL))
        
        # LOO-CV
        
        # Sparse
        adp_lasso_loo_spr = LassoLarsCV(cv=loo)
        adp_lasso_loo_spr.fit(adX_sprL, Y_spr)
        adp_lasso_loocv_mse_spr[i] += mean_squared_error(test_Y_spr, adp_lasso_loo_spr.predict(test_adX_sprL))
        
        # Dense
        adp_lasso_loo_dns = LassoLarsCV(cv=loo)
        adp_lasso_loo_dns.fit(adX_dnsL, Y_dns)
        adp_lasso_loocv_mse_dns[i] += mean_squared_error(test_Y_dns, adp_lasso_loo_dns.predict(test_adX_dnsL))

        # Calculate Ridge
        # AIC
        
        lambda_rng = np.linspace(0,100,num = 200)
        
        #Sparse
        _,__,mse = lambda_ic(lambda_rng, X, Y_spr, test_X, test_Y_spr, ic='aic')
        ridge_aic_mse_spr[i] += mse
        
        #Dense
        _,__,mse = lambda_ic(lambda_rng, X, Y_dns, test_X, test_Y_dns, ic='aic')
        ridge_aic_mse_dns[i] += mse
        
        #BIC
        #Sparse
        _,__,mse = lambda_ic(lambda_rng, X, Y_spr, test_X, test_Y_spr, ic='bic')
        ridge_bic_mse_spr[i] += mse
        
        #Dense
        _,__,mse = lambda_ic(lambda_rng, X, Y_dns, test_X, test_Y_dns, ic='bic')
        ridge_bic_mse_dns[i] += mse
        
        #LOO-CV
        
        #Sparse
        ridge_loo_spr = RidgeCV(cv = loo)
        ridge_loo_spr.fit(X,Y_spr)
        ridge_loocv_mse_spr[i] += mean_squared_error(test_Y_spr, ridge_loo_spr.predict(test_X))
        
        #Dense
        ridge_loo_dns = RidgeCV(cv = loo)
        ridge_loo_dns.fit(X,Y_dns)
        ridge_loocv_mse_dns[i] += mean_squared_error(test_Y_dns, ridge_loo_dns.predict(test_X))

        #Adaptive Ridge
        # AIC
        
        #Sparse
        _,__,mse = lambda_ic(lambda_rng,  adX_sprR, Y_spr, test_adX_sprR, test_Y_spr, ic='aic')
        adp_ridge_aic_mse_spr[i] += mse
        
        #Dense
        _,__,mse = lambda_ic(lambda_rng,  adX_dnsR, Y_dns, test_adX_dnsR, test_Y_dns, ic='aic')
        adp_ridge_aic_mse_dns[i] += mse
        
        #BIC
        #Sparse
        _,__,mse = lambda_ic(lambda_rng,  adX_sprR, Y_spr, test_adX_sprR, test_Y_spr, ic='bic')
        adp_ridge_bic_mse_spr[i] += mse.
        
        #Dense
        _,__,mse = lambda_ic(lambda_rng,  adX_dnsR, Y_dns, test_adX_dnsR, test_Y_dns, ic='bic')
        adp_ridge_bic_mse_dns[i] += mse
        
        # LOO-CV
        
        #Sparse
        adp_ridge_loo_spr = RidgeCV(cv = loo)
        adp_ridge_loo_spr.fit(adX_sprR,Y_spr)
        adp_ridge_loocv_mse_spr[i] += mean_squared_error(test_Y_spr, adp_ridge_loo_spr.predict(test_adX_sprR))
        
        #Dense
        adp_ridge_loo_dns = RidgeCV(cv = loo)
        adp_ridge_loo_dns.fit(adX_dnsR,Y_dns)
        adp_ridge_loocv_mse_dns[i] += mean_squared_error(test_Y_dns, adp_ridge_loo_dns.predict(test_adX_dnsR))

        
      


10 0
10 0.25
10 0.5
25 0
25 0.25
25 0.5
50 0
50 0.25
50 0.5


In [65]:

# Sparse Signal
EMSE = pd.DataFrame(np.r_[lasso_aic_mse_spr[None, :] ,
                            lasso_bic_mse_spr[None, :]       , 
                            lasso_loocv_mse_spr[None, :]     ,    
                            adp_lasso_aic_mse_spr[None, :]   ,     
                            adp_lasso_bic_mse_spr[None, :]   ,        
                            adp_lasso_loocv_mse_spr[None, :] ,   

                            ridge_aic_mse_spr[None, :] ,
                            ridge_bic_mse_spr[None, :] ,
                            ridge_loocv_mse_spr[None, :] , 
                            adp_ridge_aic_mse_spr[None, :] ,
                            adp_ridge_bic_mse_spr[None, :] ,
                            adp_ridge_loocv_mse_spr[None, :] , 
                            # Dense Signal

                            lasso_aic_mse_dns[None, :], 
                            lasso_bic_mse_dns[None, :] ,
                            lasso_loocv_mse_dns[None, :] , 
                            adp_lasso_aic_mse_dns[None, :] ,
                            adp_lasso_bic_mse_dns[None, :] ,
                            adp_lasso_loocv_mse_dns[None, :] , 

                            ridge_aic_mse_dns[None, :] ,
                            ridge_bic_mse_dns[None, :] ,
                            ridge_loocv_mse_dns[None, :] ,
                            adp_ridge_aic_mse_dns[None, :] ,
                            adp_ridge_bic_mse_dns[None, :] ,
                            adp_ridge_loocv_mse_dns[None, :]]) 

In [66]:
EMSE =  EMSE / datasets 

In [67]:
EMSE.columns = itertools.product([10, 25, 50],[0, 0.25, 0.5])

In [68]:
EMSE['methods'] = labs*2

In [69]:
EMSE['signal'] = ['sparse']*12 + ['dense']*12

In [70]:
EMSE

,"(10, 0)","(10, 0.25)","(10, 0.5)","(25, 0)","(25, 0.25)","(25, 0.5)","(50, 0)","(50, 0.25)","(50, 0.5)",methods,signal
0,0.045534,0.045366,0.075209,0.038493,0.157569,0.117799,0.079710,0.101904,0.277960,lasso AIC,sparse
1,0.045534,0.045366,0.075209,0.038493,0.157569,0.117799,0.079710,0.101904,0.277960,lasso BIC,sparse
2,0.045534,0.045366,0.075209,0.038493,0.157569,0.117799,0.079710,0.101904,0.277960,lasso LOO-CV,sparse
3,0.045534,0.045366,0.075209,0.038493,0.157569,0.117799,0.079710,0.101904,0.277960,adpt lasso AIC,sparse
4,0.045534,0.045366,0.075209,0.038493,0.157569,0.117799,0.079710,0.101904,0.277960,adpt lasso BIC,sparse
5,0.045534,0.045366,0.075209,0.038493,0.157569,0.117799,0.079710,0.101904,0.277960,adpt lasso LOO-CV,sparse
6,0.045061,0.008903,0.031994,0.031058,0.096670,0.043465,0.076087,0.070933,0.210572,ridge AIC,sparse
7,0.045061,0.008903,0.031994,0.031058,0.096670,0.043465,0.076087,0.070933,0.210572,ridge BIC,sparse
8,0.045543,0.045366,0.075212,0.038485,0.157596,0.117792,0.079679,0.101913,0.277980,ridge LOO-CV,sparse
9,0.045061,0.007307,0.031451,0.035534,0.100360,0.043506,0.065929,0.071748,0.086604,adpt ridge AIC,sparse


In [56]:
EMSE.to_csv('./EMSES.csv', index=False)

### (c.)

Yes we noticed general patterns. We can see that ridge methods perform better than lasso methods on dense data. While it is difficult to distinguish ridge from lasso on sparse data theoretically we know that lasso should outperform ridge. Due to time limitations we simulated 25 datasets rather than 1000 datasets. It is possible that the smaller number of datasets used to estimate our expected MSE has added noise to our results.

Another thing to note is that as the number of predictors increases the MSE for adaptive ridge increases by a much smaller rate than other methods.